In [27]:
# Ячейка для мгновенного обновления модулей
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
# libraries
import random
import pandas as pd
from itertools import permutations

from Bio import SeqIO

from word_to_biocode import WordToBiocode

import warnings 
warnings.simplefilter('ignore')

## Задача 🧬
Перевести слово (например, ФИО) в последовательность нуклеотидов (A, T, G, C)

## Решение

1. **Преобразование в двоичную систему**
    - В русском алфавите 33 буквы. Для их кодирования требуется **6 бит** (так как $2^6=64$, что достаточно для представления всех букв алфавита).
    - Каждую букву слова преобразуем в двоичный код длиной 6 цифр.

2. **Преобразование в четверичную систему**
    - Далее, этот двоичный код переводим в четверичную систему, где каждая цифра может быть 0, 1, 2 или 3.
    - Для этого каждые 2 бита двоичного кода превращаем в одну цифру четверичной системы.
    - Таким образом, *каждая буква будет представлена тремя цифрами четверичной системы*.

3. **Преобразование в нуклеотидную последовательность**
    - Каждая цифра четверичной системы (0, 1, 2, 3) кодируется нуклеотидом (например, 0 — A, 1 — C, 2 — G, 3 — T)
    - Таким образом, каждая буква слова будет закодирована в виде последовательности из 3 нуклеотидов (триплета).
    - **Длина закодированного слова** в нуклеотидах будет равна $3 \times \text{длина слова}$.

4. **Преобразование в аминокислотную последовательность**
    - Поскольку длина закодированной последовательности кратна трем, ее можно перевести в последовательность аминокислот (белок)


## Вероятность встретить закодированное слово в геноме

### Частота появления конкретной последовательности
Для случайного распределения нуклеотидов вероятность встретить определенную последовательность длиной $l$:
$$\frac{1}{4^l} = 4^{-l},$$ 

так как у каждого нуклеотида есть 4 возможных варианта (A, C, G, T)

Геном человека состоит из примерно $3.2 \cdot 10^9$ нуклеотидов. Т.е. возможных позиций для начала последовательности длиной $l$:
$3.2 \cdot 10^9 - l + 1 \approx 3.2 \cdot 10^9 \text{ н.}$

Чтобы вычислить, как часто можно встретить нашу закодированную последовательность длиной $l$ в геноме, умножаем количество возможных позиций на вероятность вообще получить такую последовательность. Тогда последовательность длиной $l$ будет встречаться в человеческом геноме со следующей частотой:
$$\frac{3.2 \cdot 10^9}{4^l} \text{ н.}$$

### Вероятность встретить последовательность хотя бы 1 раз
Вероятность встретить хотя бы одно вхождение последовательности в геноме можно выразить так: 
$$P = 1 - (1 - \frac{1}{4^l})^{3.2 \cdot 10^9}$$

## Реализация - перевод ФИО в ДНК и белок

In [30]:
sequences = []
protein_sequences = []
convertation = []

word = 'MALKOVAKE'

for nucl in list(permutations(['A', 'T', 'G', 'C'])):
    wordfinder = WordToBiocode(lang='eng', nucl=nucl)

    seq_word = wordfinder.encode(word, type_seq='DNA')
    protein_word = wordfinder.encode(word, type_seq='PROTEIN')

    sequences.append(seq_word)
    protein_sequences.append(protein_word)
    convertation.append(''.join(nucl))

    print(wordfinder.nucl_to_fourth)
    print(f'DNA: {seq_word}\nPROTEIN: {protein_word}')
    print('='*60)

{'A': '0', 'T': '1', 'G': '2', 'C': '3'}
DNA: ACAAAAAGCAGGACGTTTAAAAGGATA
PROTEIN: TKSRTFKRI
{'A': '0', 'T': '1', 'C': '2', 'G': '3'}
DNA: AGAAAAACGACCAGCTTTAAAACCATA
PROTEIN: RKTTSFKTI
{'A': '0', 'G': '1', 'T': '2', 'C': '3'}
DNA: ACAAAAATCATTACTGGGAAAATTAGA
PROTEIN: TKIITGKIR
{'A': '0', 'G': '1', 'C': '2', 'T': '3'}
DNA: ATAAAAACTACCATCGGGAAAACCAGA
PROTEIN: IKTTIGKTR
{'A': '0', 'C': '1', 'T': '2', 'G': '3'}
DNA: AGAAAAATGATTAGTCCCAAAATTACA
PROTEIN: RKMISPKIT
{'A': '0', 'C': '1', 'G': '2', 'T': '3'}
DNA: ATAAAAAGTAGGATGCCCAAAAGGACA
PROTEIN: IKSRMPKRT
{'T': '0', 'A': '1', 'G': '2', 'C': '3'}
DNA: TCTTTTTGCTGGTCGAAATTTTGGTAT
PROTEIN: SFCWSKFWY
{'T': '0', 'A': '1', 'C': '2', 'G': '3'}
DNA: TGTTTTTCGTCCTGCAAATTTTCCTAT
PROTEIN: CFSSCKFSY
{'T': '0', 'G': '1', 'A': '2', 'C': '3'}
DNA: TCTTTTTACTAATCAGGGTTTTAATGT
PROTEIN: SFY*SGF*C
{'T': '0', 'G': '1', 'C': '2', 'A': '3'}
DNA: TATTTTTCATCCTACGGGTTTTCCTGT
PROTEIN: YFSSYGFSC
{'T': '0', 'C': '1', 'A': '2', 'G': '3'}
DNA: TGTTTTTAGTAATGACCCTTTTAA

In [36]:
data = pd.DataFrame({"encoding": convertation, 
              "DNA_seq": sequences, 
              "PROTEIN_seq": protein_sequences})

data.sample(5, random_state=17)

,encoding,DNA_seq,PROTEIN_seq
10,TCAG,TGTTTTTAGTAATGACCCTTTTAATCT,CF***PF*S
0,ATGC,ACAAAAAGCAGGACGTTTAAAAGGATA,TKSRTFKRI
21,CTGA,CACCCCCGACGGCAGTTTCCCCGGCTC,HPRRQFPRL
11,TCGA,TATTTTTGATGGTAGCCCTTTTGGTCT,YF*W*PFWS
5,ACGT,ATAAAAAGTAGGATGCCCAAAAGGACA,IKSRMPKRT


## Переводим ген в слова
- В нашем алфавите нет пунктуации и пробелов
- Можно добавить их в наш словарь. Было бы логично сделать стоп-кодоны точкой, а самый часто-встречающийся кодон - пробелом.
- Можно обучить модель (н-р RNN) на парах (без пунктуации, с пунктуацией), чтобы она предсказывала, где вставить пробелы, запятые, точки и другие символы. Но смысла нет, так как декодирование генов в приницпе бессмысленно)

In [85]:
word = 'hi'
seq_to_word = WordToBiocode(lang='eng')

assert word.lower() == seq_to_word.decode(seq_to_word.encode(word))

In [87]:
seq_to_word.encode(word)

'ATCAGA'

## Читаем .fasta и декодируем различные гены

In [88]:
def read_fasta(path_to_fasta: str):
    sequence = []
    for record in SeqIO.parse(path_to_fasta, "fasta"):
        sequence.append(str(record.seq))
    sequence = ''.join(sequence)
    return sequence

### APOE

In [138]:
apoe_seq = read_fasta('genes/APOE.fasta')
apoe_word = seq_to_word.decode(apoe_seq)
apoe_word[:100]

'llcjksaksltwikplipscllknnhfrnzzhenmsinuaybsebmunkwigykbnngkfypfxreiasgazfkwegwqkkjwfjxxibkfkrhppivhe'

### HBB

In [139]:
hbb_seq = read_fasta('genes/HBB.fasta')
hbb_word = seq_to_word.decode(hbb_seq)
hbb_word[:100]

'mvmnxnldamhykijuzkyogcwkrtcmukienmcnvenwrvpktpkxvvngdcccavjdcpvmjzcdxkjyzvvujzkcejdkvigdigfcvjxvrwxe'

### BRCA1

In [140]:
brca_word = seq_to_word.decode(read_fasta('genes/BRCA1.fasta'))
brca_word[:100]

'nokxdkkadicafrxwcctagugiasjjykdjpmrvbvydxpiicjrigeqatcaudwwclryavxlwmjgmefwakaendhqcgcvampaxtidahkad'

### TP53

In [141]:
tp53_word = seq_to_word.decode(read_fasta('genes/TP53.fasta'))
tp53_word[:100]

'ajkvxvyisdfxignnyalbyvrymgyikiylbiplltpplhxzxdvxlzwwnzyxpmzyisrjkavzvyavmszyyytqxniqnoyzwiiikkqlnapn'

### FTO

In [142]:
fto_word = seq_to_word.decode(read_fasta('genes/FTO.fasta'))
fto_word[:100]

'lzpfqhhvnkbliinmkkhtvjlgcpnczilcpkvgakkwkeugufwwabcpxxqfzztfiyytvegyiybhzlzwwydjwzueyhvdtftyzjgyqqiv'

плохая шутка - даже ген напоминает о больном:

In [143]:
'fat' in fto_word

True

## Поиск слов в декодированном гене

- `nltk.corpus.words` 
- **Алгоритм Кнута-Морриса-Пратта** (KMP) или **Алгоритм Бойера-Мура** (Boyer-Moore) можно использовать для поиска конкретных слов в длинных строках. Эти алгоритмы оптимизированы для быстрого поиска подстрок и могут быть полезны при поиске слов в случайной последовательности.


In [144]:
import re
import nltk
from nltk.corpus import words

# nltk.download('words')

def find_words(sequence, word_list, min_length=1, max_length=float('inf'), ignore_punctuation=True):
    found_words = []
    
    # пунктуации сейчас все равно нет, но мало ли добавлю
    if ignore_punctuation:
        sequence = re.sub(r'[^\w\s]', '', sequence)

    for word in word_list:
        if min_length <= len(word) <= max_length:
            if re.search(word, sequence):
                found_words.append(word)

    return found_words

In [145]:
word_list = set(words.words())
print(len(word_list))

235892


In [151]:
%%time
apoe_found = find_words(apoe_word, word_list, min_length=3)
apoe_found[:20]

CPU times: user 3.74 s, sys: 0 ns, total: 3.74 s
Wall time: 4.15 s


['pub',
 'wun',
 'lip',
 'fig',
 'six',
 'wig',
 'lin',
 'poky',
 'kil',
 'yoi',
 'hen',
 'sin',
 'kip',
 'tab',
 'lit',
 'mun',
 'vau',
 'inn',
 'gaz',
 'linn']

In [152]:
%%time
tp53_found = find_words(tp53_word, word_list, min_length=3)
tp53_found[:20]

CPU times: user 3.63 s, sys: 0 ns, total: 3.63 s
Wall time: 3.99 s


['beg',
 'yis',
 'thy',
 'noy',
 'kat',
 'alb',
 'aka',
 'yalb',
 'ani',
 'nap',
 'pua',
 'urf']

In [153]:
%%time
hbb_found = find_words(hbb_word, word_list, min_length=3)
hbb_found[:20]

CPU times: user 3.63 s, sys: 0 ns, total: 3.63 s
Wall time: 3.99 s


['wem', 'arx', 'nix', 'ewe', 'dam', 'dig', 'baa', 'aka', 'dak', 'ary']

In [154]:
%%time
brca_found = find_words(brca_word, word_list, min_length=3)
print(len(brca_found))

98
CPU times: user 3.85 s, sys: 153 μs, total: 3.85 s
Wall time: 4.16 s


In [155]:
sorted(brca_found, key=len, reverse=True)[:20]

['pali',
 'clue',
 'cusk',
 'bead',
 'lied',
 'waka',
 'wice',
 'puka',
 'gleg',
 'masa',
 'vamp',
 'cay',
 'met',
 'poh',
 'dab',
 'day',
 'gim',
 'yan',
 'ked',
 'pax']

In [156]:
%%time
fto_found = find_words(fto_word, word_list, min_length=3)
print(len(fto_found))

1476
CPU times: user 12.8 s, sys: 10.2 ms, total: 12.9 s
Wall time: 11.3 s


FTO просто мем:

In [157]:
sorted(fto_found, key=len, reverse=True)[:20]

['urushi',
 'laical',
 'revere',
 'refer',
 'idiot',
 'birsy',
 'varve',
 'kilah',
 'cymba',
 'diver',
 'blind',
 'black',
 'stiff',
 'rever',
 'spewy',
 'abura',
 'remex',
 'mafic',
 'fusee',
 'abear']